# Task-1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from scipy.stats import norm
from jugaad_data.nse import NSELive

### Code for extracting option chain from NSE data and save it in PC storage
### (Data used is of 23-May-2024)

In [2]:
# n = NSELive()
# option_chain = n.index_option_chain("NIFTY")
# # option_chain

# current_price = 0
# if 'CE' in option_chain['records']['data']:
#     current_price = option_chain['records']['data'][0]['CE']['underlyingValue']
# else:
#     current_price = option_chain['records']['data'][0]['PE']['underlyingValue']
# # current_price    

# strikePrice = []
# impliedVolatility = []
# lastPrice = []
# empty_int = []
# empty_str = []
# date = '23-May-2024'
# expiry_date = '27-Jun-2024'
# date = []
# expiryDate = []
# underlyingValue=[]
# for record in option_chain['records']['data']:
#     if 'CE' in record:
#         if record['CE']['expiryDate'] == expiry_date:
#             strikePrice.append(record['CE']['strikePrice'])
#             impliedVolatility.append(record['CE']['impliedVolatility'])
#             lastPrice.append(record['CE']['lastPrice'])
#             empty_int.append(0.0)
#             empty_str.append('-')
#             date.append(date)
#             expiryDate.append(expiry_date)
#             underlyingValue.append(current_price)
# data=pd.DataFrame({'date':date,'expiryDate':expiryDate,'underlyingValue':underlyingValue,'strikePrice':strikePrice,'impliedVolatility':impliedVolatility,
                    # 'lastPrice':lastPrice,'bsm_model_price':empty_int,'moneyness':empty_str,'underpriced/overpriced':empty_str,'buy/sell':empty_str})

# data = data[data.impliedVolatility>0]
# data = data.reset_index(drop=True)
# data = data.sort_values(by='strikePrice')

# data.to_csv('task_1_options_data.csv', index=False)

### Extracting data from csv file

In [3]:
data = pd.read_csv('task_1_options_data.csv')

In [4]:
data

,date,expiryDate,underlyingValue,strikePrice,impliedVolatility,lastPrice,bsm_model_price,moneyness,underpriced/overpriced,buy/sell
0,23-May-2024,27-Jun-2024,22805.3,21000,17.14,2060.00,0.0,-,-,-
1,23-May-2024,27-Jun-2024,22805.3,21100,16.42,1948.40,0.0,-,-,-
2,23-May-2024,27-Jun-2024,22805.3,21500,18.84,1601.35,0.0,-,-,-
3,23-May-2024,27-Jun-2024,22805.3,21600,17.76,1503.00,0.0,-,-,-
4,23-May-2024,27-Jun-2024,22805.3,21700,17.96,1400.00,0.0,-,-,-
...,...,...,...,...,...,...,...,...,...,...
58,23-May-2024,27-Jun-2024,22805.3,24450,17.52,87.25,0.0,-,-,-
59,23-May-2024,27-Jun-2024,22805.3,24500,17.54,85.20,0.0,-,-,-
60,23-May-2024,27-Jun-2024,22805.3,25000,18.53,48.60,0.0,-,-,-
61,23-May-2024,27-Jun-2024,22805.3,26000,20.99,19.05,0.0,-,-,-


#### Determining moneyness of the option

In [7]:
f=0
current_price = data.loc[0,'underlyingValue']
for index,row in data.iterrows():
    if row['strikePrice']<current_price:
        data.loc[index,'moneyness'] = 'ITM'
    else:
        if f==0:
            f=1
            if index==0:
                data.loc[index,'moneyness'] = 'ATM'
            elif data.loc[index,'strikePrice']-current_price < current_price-data.loc[index-1,'strikePrice']:
                data.loc[index,'moneyness'] = 'ATM'
            else:
                data.loc[index,'moneyness'] = 'OTM'
                data.loc[index-1,'moneyness'] = 'ATM'
        else: data.loc[index,'moneyness'] = 'OTM'

#### Determining time left till expiry

In [8]:
date = data.loc[0,'date']
expiry_date = data.loc[0,'expiryDate']
date1 = datetime.datetime.strptime(date, '%d-%b-%Y').date()
date2 = datetime.datetime.strptime(expiry_date, '%d-%b-%Y').date()
difference = date2 - date1
time_to_expiry = difference.days/365
risk_free_interest_rate = 0.1

### Black-Scholes Model

In [9]:
def estimated_price(s, v,c=current_price, t=time_to_expiry, r=risk_free_interest_rate, option_type='call'):
    v/=100
    d1 = (np.log(c/s) + (r + 0.5 * (v ** 2)) * t) / (v * np.sqrt(t))
    d2 = d1 - v * np.sqrt(t)
    if option_type == 'call': estimated_price = c * norm.cdf(d1) - s * np.exp(-r * t) * norm.cdf(d2)
    else: estimated_price = s * np.exp(-r * t) * norm.cdf(-d2) - c * norm.cdf(-d1)
    return round(estimated_price, 2)   

### Determining if option should be bought or sold

In [10]:
for index,row in data.iterrows():
    data.loc[index,'bsm_model_price'] = estimated_price(row['strikePrice'], row['impliedVolatility'])
for index,row in data.iterrows():
    if row['bsm_model_price']<=row['lastPrice']:
        data.loc[index,'underpriced/overpriced'] = 'overpriced'
        data.loc[index,'buy/sell'] = 'sell'
    else:
        data.loc[index,'underpriced/overpriced'] = 'underpriced'
        data.loc[index,'buy/sell'] = 'buy'

In [11]:
pd.set_option('display.max_rows', None)
data

,date,expiryDate,underlyingValue,strikePrice,impliedVolatility,lastPrice,bsm_model_price,moneyness,underpriced/overpriced,buy/sell
0,23-May-2024,27-Jun-2024,22805.3,21000,17.14,2060.00,2025.13,ITM,overpriced,sell
1,23-May-2024,27-Jun-2024,22805.3,21100,16.42,1948.40,1926.13,ITM,overpriced,sell
2,23-May-2024,27-Jun-2024,22805.3,21500,18.84,1601.35,1586.42,ITM,overpriced,sell
3,23-May-2024,27-Jun-2024,22805.3,21600,17.76,1503.00,1485.10,ITM,overpriced,sell
4,23-May-2024,27-Jun-2024,22805.3,21700,17.96,1400.00,1402.74,ITM,underpriced,buy
5,23-May-2024,27-Jun-2024,22805.3,21750,10.11,1247.00,1272.29,ITM,underpriced,buy
6,23-May-2024,27-Jun-2024,22805.3,21800,18.64,1346.50,1331.20,ITM,overpriced,sell
7,23-May-2024,27-Jun-2024,22805.3,21850,18.13,1260.00,1281.65,ITM,underpriced,buy
8,23-May-2024,27-Jun-2024,22805.3,21900,18.58,1262.10,1249.83,ITM,overpriced,sell
9,23-May-2024,27-Jun-2024,22805.3,21950,14.87,1121.75,1144.12,ITM,underpriced,buy
